In [1]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pickle
import numpy as np


In [2]:
with open('finalized_model.pkl', 'rb') as f:
    model = pickle.load(f)

C:\Users\Jonathan\anaconda3\envs\recommendation-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_keys(path):
    with open("/Users/Jonathan/Documents/Flatiron/phase_5/P5_spotify_recommendations/.secret/spotify_api.json") as f:
        return json.load(f)

In [4]:
keys = get_keys("/Users/Jonathan/.secret/spotify_api.json")

client = keys['client']
api_key = keys['api_key']

In [5]:
auth_manager = SpotifyClientCredentials(client_id = client, client_secret = api_key)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
# Importing data
spot_df = pd.read_csv('data/spotify_playlist.csv')
spot_df.head()

,artist,artist_id,popularity,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bon Jovi,58lV9VcRSjABbAbfWS6skp,8,Slippery When Wet,Livin' On A Prayer,0J6mQxEZnlRt9ymzFntA6z,0.534,0.887,0,-3.777,1,0.0345,0.000099,0.325,0.720,122.494,249293,4
1,Bon Jovi,58lV9VcRSjABbAbfWS6skp,0,Bon Jovi Greatest Hits,Wanted Dead Or Alive,4Zj9VM4fO1KwkU3lFaECsW,0.257,0.819,7,-3.562,1,0.0422,0.015000,0.324,0.267,150.354,308560,4
2,Queen,1dfeR4HaWDbWqFHLkxsg1d,64,News Of The World (Deluxe Remastered Version),We Are The Champions - Remastered 2011,7ccI9cStQbQdystvc6TvxD,0.268,0.459,7,-6.948,0,0.0346,0.000000,0.119,0.172,64.223,179200,4
3,Queen,1dfeR4HaWDbWqFHLkxsg1d,69,News Of The World (Deluxe Remastered Version),We Will Rock You - Remastered 2011,54flyrjcdnQdco7300avMJ,0.693,0.497,2,-7.316,1,0.1190,0.000000,0.258,0.473,81.308,122067,4
4,Bon Jovi,58lV9VcRSjABbAbfWS6skp,0,Cross Road,You Give Love A Bad Name,7LBJui5MdjfCd8YZr4xaqA,0.556,0.945,0,-4.347,0,0.0496,0.000004,0.394,0.781,122.875,224307,4


In [7]:
def get_song(song, artist):
    
    playlist_features_list = ["artist", "artist_id", "popularity", "album", "track_name", "track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]

    song_df  = pd.DataFrame(columns = playlist_features_list)
    song = sp.search(q = 'track: {},  artist: {}'.format(song, artist), limit=1)
    for track in song:
    
        playlist_features = {}
        playlist_features["artist"] = song['tracks']['items'][0]['artists'][0]['name']
        playlist_features['artist_id'] = song['tracks']['items'][0]['artists'][0]['id']
        playlist_features['popularity'] = song['tracks']['items'][0]['popularity']
        playlist_features["album"] = song['tracks']['items'][0]['album']['name']
        playlist_features["track_name"] = song['tracks']['items'][0]['name']
        playlist_features["track_id"] = song['tracks']['items'][0]['id']

        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        song_df = pd.concat([song_df, track_df], ignore_index = True)
        return song_df

In [8]:
def predict(song_title, artist, df):
    song = get_song(song_title, artist)
    new_df = pd.concat([song, spot_df], ignore_index=True)
    new_df_feat = new_df.select_dtypes(np.number)
    preds = model.fit_predict(new_df_feat)
    coords = model.named_steps['umap'].transform(new_df_feat)
    new_df['cluster'] = preds
    new_df['x'], new_df['y'] = [x[0] for x in coords], [x[1] for x in coords]
    cluster = new_df.loc[new_df['track_name'] == new_df['track_name'][0], 'cluster'].to_list()[0]
    new_df = new_df.loc[new_df['cluster'] == cluster]
    
    
    return new_df

In [9]:
def dist(row):
    x = row['x']
    y = row['y']
    distance=np.sqrt((xt-x)**2 + (yt-y)**2)
    return distance
    

In [19]:
new_df = predict('jimmy cooks', 'drake', spot_df)

C:\Users\Jonathan\anaconda3\envs\recommendation-env\lib\site-packages\sklearn\cluster\_kmeans.py:1848: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


In [20]:
xt = new_df.loc[new_df['track_name'] == new_df['track_name'][0],'x']
yt = new_df.loc[new_df['track_name'] == new_df['track_name'][0],'y']

new_df['distance'] = new_df.apply(dist, axis =1)
new_df.sort_values('distance')[0:10]
    

,artist,artist_id,popularity,album,track_name,track_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,x,y,distance
0,Drake,3TVXtAsR1Inumwj472S9r4,91,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,...,0.000002,0.0930,0.366,165.921,218365,4,0,10.397751,-1.746101,0.000000
6360,Petter,5A0Bu9azuFEnud3q7t0V2r,53,Skeppsholmen,Se på mig nu (feat. Linnea Henriksson) - Singe...,7rklm17yoayxw45lpzv5uv,0.622,0.729,0,-7.929,...,0.000006,0.1630,0.532,106.053,250040,4,0,10.387798,-1.727138,0.021417
6269,Rhett and Link,3XKtycltpy1gaeny0Bdzcs,26,Get off the Phone,Get off the Phone,7q6GzP2EUCZjGNYkpZtaie,0.702,0.624,11,-8.440,...,0.000003,0.3850,0.519,95.993,177659,4,0,10.424678,-1.749789,0.027178
131,KISS,07XSN3sPlIlB2L2XNcTwJw,67,Animalize,Heaven's On Fire,1brpdmqkx3kSxyqzqXfW7J,0.679,0.866,0,-6.053,...,0.000031,0.3120,0.785,124.324,201400,4,0,10.402135,-1.713354,0.033039
5171,Zimm,3LxxdwGQuhVoFJPVEKX0Pk,36,Problems,Problems,6YSBr8S0cEIKzIPSADDidc,0.591,0.767,7,-5.897,...,0.000000,0.1140,0.509,96.684,195000,4,0,10.390154,-1.712055,0.034884
2899,Suicidal Tendencies,3WPKDlucMsXH6FC1XaclZC,58,Suicidal Tendencies,Institutionalized,7G83ZwD2msTtFaBQExc8QB,0.358,0.804,0,-8.534,...,0.000000,0.1110,0.625,91.842,229960,3,0,10.371666,-1.771016,0.036071
3187,The Presidents Of The United States Of America,1lZvg4fNAqHoj6I9N8naBM,49,The Presidents of The United States of America...,Kitty,4WamMWxFi6ts3xkIhivF5c,0.498,0.801,1,-5.919,...,0.000000,0.0510,0.550,92.126,203133,4,0,10.410600,-1.711519,0.036893
5344,Macklemore & Ryan Lewis,5BcAKTbp20cv7tC5VqPFoC,1,The Heist,Thrift Shop (feat. Wanz),4YMqbFcDIFiCBd02PzUBcM,0.781,0.526,6,-6.985,...,0.000000,0.0457,0.662,94.992,235613,4,0,10.424851,-1.720822,0.037061
3930,Avicii,1vCWHaC5f2uS3yhpwWbIA6,85,The Days / Nights,The Nights,0ct6r3EGTcMLPtrXHDvVjc,0.527,0.835,6,-5.298,...,0.000000,0.2490,0.654,125.983,176658,4,0,10.380046,-1.781879,0.039918
254,The Hollies,6waa8mKu91GjzD4NlONlNJ,42,Midas Touch - The Very Best of the Hollies,Bus Stop - 1997 Remaster,0K3i1MJwU4v0Aq0VdYZnPg,0.625,0.760,4,-5.591,...,0.000000,0.1160,0.697,136.541,175160,4,0,10.429250,-1.720233,0.040760
